In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from plotly.offline import plot as off
import plotly.figure_factory as ff
import plotly.io as pio
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import Dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
df=pd.read_csv('/content/drive/MyDrive/DS Data Sets/KNN/Zoo.csv')

In [7]:
df

,animal name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [9]:
df['animal name'].value_counts()

frog        2
pony        1
sealion     1
seal        1
seahorse    1
           ..
gorilla     1
goat        1
gnat        1
girl        1
wren        1
Name: animal name, Length: 100, dtype: int64

In [12]:
df['animal name'].unique()

array(['aardvark', 'antelope', 'bass', 'bear', 'boar', 'buffalo', 'calf',
       'carp', 'catfish', 'cavy', 'cheetah', 'chicken', 'chub', 'clam',
       'crab', 'crayfish', 'crow', 'deer', 'dogfish', 'dolphin', 'dove',
       'duck', 'elephant', 'flamingo', 'flea', 'frog', 'fruitbat',
       'giraffe', 'girl', 'gnat', 'goat', 'gorilla', 'gull', 'haddock',
       'hamster', 'hare', 'hawk', 'herring', 'honeybee', 'housefly',
       'kiwi', 'ladybird', 'lark', 'leopard', 'lion', 'lobster', 'lynx',
       'mink', 'mole', 'mongoose', 'moth', 'newt', 'octopus', 'opossum',
       'oryx', 'ostrich', 'parakeet', 'penguin', 'pheasant', 'pike',
       'piranha', 'pitviper', 'platypus', 'polecat', 'pony', 'porpoise',
       'puma', 'pussycat', 'raccoon', 'reindeer', 'rhea', 'scorpion',
       'seahorse', 'seal', 'sealion', 'seasnake', 'seawasp', 'skimmer',
       'skua', 'slowworm', 'slug', 'sole', 'sparrow', 'squirrel',
       'starfish', 'stingray', 'swan', 'termite', 'toad', 'tortoise',
       

In [14]:
df.type.unique()

array([1, 4, 2, 7, 6, 5, 3])

In [16]:
df.type.value_counts()

1    41
2    20
4    13
7    10
6     8
3     5
5     4
Name: type, dtype: int64

In [18]:
df2=df.iloc[:,1:]
df2

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [20]:
x=df2.iloc[:,0:16]
y=df2.iloc[:,-1]

In [22]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.33,random_state=35)

In [24]:
# Train the K-Neighbors Classifier without Hyper-parameter Tuning

In [26]:
k=np.sqrt(df2.shape[0]/2)
k

7.106335201775948

In [74]:
knn=KNeighborsClassifier(n_neighbors=7)
model=knn.fit(x_train,y_train)

In [30]:
model.score(x_train,y_train)

0.8805970149253731

In [32]:
y_pred=model.predict(x_test)
y_pred

array([6, 1, 3, 1, 2, 1, 1, 6, 6, 1, 4, 2, 2, 1, 1, 1, 2, 2, 1, 4, 1, 6,
       4, 5, 5, 1, 4, 5, 4, 2, 2, 4, 6, 6])

In [34]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.91      0.83      0.87        12
           2       1.00      1.00      1.00         7
           3       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         3
           5       0.33      1.00      0.50         1
           6       0.67      1.00      0.80         4
           7       0.00      0.00      0.00         6

    accuracy                           0.74        34
   macro avg       0.49      0.69      0.55        34
weighted avg       0.66      0.74      0.68        34



In [36]:
# Train the K-Neighbors Classifier with Hyper-parameter Tuning

In [38]:
# Use GridsearchCV

In [71]:
knnc=KNeighborsClassifier()
param_grid=dict(n_neighbors=range(1,7))

In [43]:
model=GridSearchCV(knnc,param_grid,refit=True,verbose=3)

In [45]:
model.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.929 total time=   0.0s
[CV 2/5] END .....................n_neighbors=1;, score=1.000 total time=   0.0s
[CV 3/5] END .....................n_neighbors=1;, score=0.923 total time=   0.0s
[CV 4/5] END .....................n_neighbors=1;, score=0.923 total time=   0.0s
[CV 5/5] END .....................n_neighbors=1;, score=0.846 total time=   0.0s
[CV 1/5] END .....................n_neighbors=2;, score=0.857 total time=   0.0s
[CV 2/5] END .....................n_neighbors=2;, score=0.929 total time=   0.0s
[CV 3/5] END .....................n_neighbors=2;, score=0.923 total time=   0.0s
[CV 4/5] END .....................n_neighbors=2;, score=0.923 total time=   0.0s
[CV 5/5] END .....................n_neighbors=2;, score=0.846 total time=   0.0s
[CV 1/5] END .....................n_neighbors=3;, score=0.857 total time=   0.0s
[CV 2/5] END .....................n_neighbors=3;,

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 5)}, verbose=3)

In [47]:
model.best_params_

{'n_neighbors': 1}

In [49]:
model.best_estimator_

KNeighborsClassifier(n_neighbors=1)

In [51]:
model.score(x_train,y_train)

1.0

In [53]:
# Predictions using test data
y_pred=model.predict(x_test)
y_pred

array([6, 5, 7, 1, 2, 1, 1, 6, 6, 1, 7, 2, 2, 1, 1, 1, 2, 2, 1, 4, 1, 7,
       1, 5, 7, 1, 4, 5, 4, 2, 2, 1, 7, 6])

In [55]:
# print classification report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         7
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         3
           5       0.33      1.00      0.50         1
           6       1.00      1.00      1.00         4
           7       1.00      0.83      0.91         6

    accuracy                           0.94        34
   macro avg       0.76      0.83      0.77        34
weighted avg       0.95      0.94      0.94        34



In [57]:
# Visualizing CV results

In [72]:
k_range = range(1, 7)
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x, y, cv=5)
    k_scores.append(scores.mean())
    k_scores.append(scores.mean())

In [ ]:
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross Validated Accuracy')
plt.show()


In [ ]:
plt.figure(figsize=(10,6))

error _rate=[]
for i in range(1,7)

knn=KNeighborsClassifier(n_neighbors=i)
knn.fit(x,train,y_train)
pred_i=knn.predict(x_test)
error_rate.append(np.mean(pred_i !=y_test))

plt.plot(range(1,7),error_rate, color = 'pink', linestyle='dashed',marker='0'
            markerfacecolor='blue',markersize=12)

plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')